In [2]:
!pip install chess

     |████████████████████████████████| 149 kB 2.7 MB/s eta 0:00:01


In [1]:
from chessdotcom import get_player_game_archives, get_leaderboards, get_player_stats
import chess
import requests
import chessdotcom
from queue import Queue

In [2]:
class Player():
    def __init__(self, name, color, elo):
        self.name = name
        self.color = color
        self.elo = elo

In [3]:
class Game():
    def __init__(self, player1, player2, game, winner):
        self.player1 = player1
        self.player2 = player2
        self.game = game
        self.winner = winner

## Retrieve Games within a Certain ELO Range

In [69]:
def get_data(elo_lb, elo_ub, num_players_cap, num_games_cap):
    '''
    performs a bfs to get game data from a list of users of a desired elo.
    elo_lb: lower bound.
    elo_ub: upper bound..
    num_players_cap: number of players we want to cap our requests to.
    num_games_cap: number of games per player we want to cap our exploration at.
    '''

    games_lst = [] # populate a list of games.


    # Initialize a Queue
    user_list = Queue()
    user_list.put("fabianocaruana")

    # Mark the start node as visited
    visited = {"fabianocaruana"}

    # Loop until the queue is empty
    while len(visited) < num_players_cap: # stop when we have requested num_players_cap players.

        # Dequeue a vertex from the queue
        curr_user = user_list.get()

        # print(curr_user)

        # Request player games from server.
        requested_player = get_player_game_archives(curr_user)
        
        # print(requested_player) # stuck on requests
        
        this_playergames = requests.get(url=requested_player.json["archives"][0]).json()["games"]

        # Add all adjacent vertices to the queue
        for i in range(min(len(this_playergames), num_games_cap)): # loop through the first num_games_cap games
            opponent_elo = this_playergames[i]['black']['rating']
            opponent_user = this_playergames[i]['black']['username']

            if opponent_elo >= elo_lb and opponent_elo <= elo_ub and opponent_user not in visited:

                games_lst.append(this_playergames[i])

                visited.add(opponent_user)
                user_list.put(opponent_user)

    return (games_lst, visited)

In [70]:
print(visited)

{'Checkbrot', 'mliccione', 'samburu4', 'arnold_torres', 'fabianocaruana', 'srikrishna143', 'Donderiban', 'mikel59kos', 'Emilvas', 'alicactus1980', 'don2_r2d2', 'Pleven', 'faltoe', 'mastroboz', 'Rottenhogin', 'DIAMONDSTORM', 'SirReini', 'bimbim2', 'jasonnyc', 'Aboy24_vs_World', 'alexanderthelittle', 'FabianoCaruana', 'Barefoot_Player', 'Wander92', 'Dalnoboishik', 'Boring_Bloggins'}


In [71]:
test6 = get_data(elo_lb=1400, elo_ub=1600, num_players_cap=10, num_games_cap=50)
test6[1]

{'Barefoot_Player',
 'Boring_Bloggins',
 'FabianoCaruana',
 'Rottenhogin',
 'SirReini',
 'Wander92',
 'alexanderthelittle',
 'fabianocaruana',
 'faltoe',
 'jasonnyc',
 'mastroboz',
 'mliccione',
 'srikrishna143'}

In [7]:
components = games.json()["games"][0]["pgn"].split("\n")
components

['[Event "Live Chess"]',
 '[Site "Chess.com"]',
 '[Date "2013.03.17"]',
 '[Round "-"]',
 '[White "FabianoCaruana"]',
 '[Black "hastily"]',
 '[Result "1-0"]',
 '[CurrentPosition "3r4/p6p/Q2b2p1/3Rk2N/P1p1P3/5P2/1P2K1PP/7R b - -"]',
 '[Timezone "UTC"]',
 '[ECO "D10"]',
 '[ECOUrl "https://www.chess.com/openings/Slav-Defense-3.Nc3-dxc4"]',
 '[UTCDate "2013.03.17"]',
 '[UTCTime "15:16:14"]',
 '[WhiteElo "1363"]',
 '[BlackElo "1193"]',
 '[TimeControl "180"]',
 '[Termination "FabianoCaruana won by resignation"]',
 '[StartTime "15:16:14"]',
 '[EndDate "2013.03.17"]',
 '[EndTime "15:19:21"]',
 '[Link "https://www.chess.com/game/live/474897192"]',
 '',
 '1. d4 {[%clk 0:03:00]} 1... d5 {[%clk 0:03:00]} 2. c4 {[%clk 0:02:57.8]} 2... dxc4 {[%clk 0:02:57.2]} 3. Nc3 {[%clk 0:02:56.9]} 3... c6 {[%clk 0:02:56.5]} 4. a4 {[%clk 0:02:54.8]} 4... Nf6 {[%clk 0:02:52.7]} 5. e4 {[%clk 0:02:53.5]} 5... c5 {[%clk 0:02:48.1]} 6. d5 {[%clk 0:02:51.5]} 6... Bg4 {[%clk 0:02:44]} 7. f3 {[%clk 0:02:49.7]} 7... Bh5 {[

In [7]:
ldrboard = get_leaderboards().json # dictionary containing leaderboard (top 50 players).
daily_players = ldrboard["leaderboards"]["daily"]
daily_players

[{'player_id': 2305524,
  '@id': 'https://api.chess.com/pub/player/zgorl',
  'url': 'https://www.chess.com/member/Zgorl',
  'username': 'Zgorl',
  'score': 2624,
  'rank': 1,
  'country': 'https://api.chess.com/pub/country/NL',
  'title': 'FM',
  'status': 'premium',
  'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/2305524.5341b605.200x200o.67a89f6c51b4.jpeg',
  'trend_score': {'direction': 1, 'delta': 9},
  'trend_rank': {'direction': 0, 'delta': 0},
  'flair_code': 'diamond_traditional',
  'win_count': 321,
  'loss_count': 204,
  'draw_count': 45},
 {'player_id': 1448848,
  '@id': 'https://api.chess.com/pub/player/the_evil_ducklings',
  'url': 'https://www.chess.com/member/The_Evil_Ducklings',
  'username': 'The_Evil_Ducklings',
  'score': 2530,
  'rank': 2,
  'country': 'https://api.chess.com/pub/country/US',
  'title': 'FM',
  'name': 'Roger LaFlair',
  'status': 'premium',
  'avatar': 'https://images.chesscomfiles.com/uploads/v1/user/1448848.309d598e.200x200o.91897c97

In [8]:
(lb_elo, ub_elo) = (2000, 2500)
usernames = []
for player in daily_players:
    if player['score'] <= ub_elo and  player['score'] >= lb_elo:
        usernames.append(player["username"])
        
usernames

['Rainbow68',
 'dumplings',
 'cschess',
 'Francesco111',
 'MartinMatthiesen',
 'BolivarGonzalez',
 'yelwin270',
 'evgiz0r',
 'francisbegbie',
 'tax77',
 'Castlecard',
 'ALBERT_KLOC',
 'jorquerino',
 'BjornAhlander',
 'Kiekeboef',
 'Rile-Man',
 'mrf8x8',
 'Yuwono',
 'ennemyofthesystem',
 'Jdain',
 'YoAdrian76',
 'profofchess',
 'suit0flights',
 'kreismyr',
 'AndersNM',
 'Guanchess',
 'laiciffOneslraCsungaM',
 'zytsb',
 'owl1867',
 'Daat',
 'DeTimmerman',
 'chispa7777',
 'kesc',
 'danielfeinauer',
 'Kingplot65',
 'Staunton1851',
 'Slimfast023',
 'Milchone1',
 'pawnsnotdead',
 'viktorskliarov',
 'guywithbadusername',
 'IMUffeVinther-Schou',
 'RIPOL',
 'KingSize76',
 'gvh']

## Get the List of Desired Games

In [ ]:
games_lst = []

for p in player_1500s:
    response = get_player_game_archives(p)
    gamezero = requests.get(url=response.json["archives"][0]) # the first game by the player
    games_lst.append(gamezero)

Way to get games:
1) Find any 1500ish player
2) Get a random game
3) Go into their opponents account and get a random game
4) Repeat
5) Compile all of these games to make a set

Things to consider:
- Ratings are wrong when accounts are new or have low game numbers
- Pick games from unique players
- Python chess has opening and endgame databases
https://python-chess.readthedocs.io/en/latest/

Action Items:
- Parse PGN
- Find an efficient way to gather data
    - Make the dataset of PGNs
- Compute time differences between moves
- Make a "stage of game" variable to classify opening/mid/endgame
- EDA of times. Summary statistics of time vs stage of game, etc

Observations:
- Shows opening URL. Could help in game stage classification
- Current Board is a thing
